In [109]:
import pickle
import nltk
import math

In [110]:
class Docs:

    def __init__(self,Lancaster=True):

        docs = []
        self.maxfreq = 0
        MotsVides = nltk.corpus.stopwords.words('english')

        files = ["D1.txt","D2.txt","D3.txt","D4.txt"]
        file = ""
        my_dict = {}
        for i in range(len(files)):
            file = open(files[i]).read()
            file = file.replace("\n","")
            if Lancaster :
                ExpReg = nltk.RegexpTokenizer('(?:[A-Z]\.)+|\d+(?:\.\d+)?DA?|\w+|\.{3}') # \d : équivalent à [0-9] >>> 
                Termes = ExpReg.tokenize(file) 
                TermesSansMotsVides = [terme for terme in Termes if terme.lower() not in MotsVides]
                Lancaster = nltk.LancasterStemmer()
                TermesNormalisation = [Lancaster.stem(terme) for terme in TermesSansMotsVides]
            else:
                TermesNormalisation = file.split(" ")

            for term in TermesNormalisation:
                
                if (term,i) in my_dict:
                    my_dict[(term,i)] += 1
                else:
                    my_dict[(term,i)] = 1

        self.dict = my_dict

        for keys in self.dict.keys():
            if keys[1] not in docs:
                docs.append(keys[1])
            if self.dict[keys] > self.maxfreq:
                self.maxfreq = self.dict[keys]

        self.N = len(docs)
    
    def rebuild(self,Lancaster=True):
        docs = []
        self.maxfreq = 0
        MotsVides = nltk.corpus.stopwords.words('english')

        files = ["D1.txt","D2.txt","D3.txt","D4.txt"]
        file = ""
        my_dict = {}
        for i in range(len(files)):
            file = open(files[i]).read()
            file = file.replace("\n","")
            if Lancaster :
                ExpReg = nltk.RegexpTokenizer('(?:[A-Z]\.)+|\d+(?:\.\d+)?DA?|\w+|\.{3}') # \d : équivalent à [0-9] >>> 
                Termes = ExpReg.tokenize(file) 
                TermesSansMotsVides = [terme for terme in Termes if terme.lower() not in MotsVides]
                Lancaster = nltk.LancasterStemmer()
                TermesNormalisation = [Lancaster.stem(terme) for terme in TermesSansMotsVides]
            else:
                TermesNormalisation = file.split(" ")

            for term in TermesNormalisation:
                
                if (term,i) in my_dict:
                    my_dict[(term,i)] += 1
                else:
                    my_dict[(term,i)] = 1

        self.dict = my_dict

        for keys in self.dict.keys():
            if keys[1] not in docs:
                docs.append(keys[1])
            if self.dict[keys] > self.maxfreq:
                self.maxfreq = self.dict[keys]

        self.N = len(docs)
        self.DictWeight()

    def calcWeight(self,word,doc):

        inDocs = []

        for keys in self.dict.keys():


            if keys[0] == word:
                if keys[1] not in inDocs:
                    inDocs.append(keys[1])

        # nombre de documents contenant le mot
        Ni = len(inDocs)
        # nombre de documents

        try:
            tf = self.dict[(word,doc)]/self.maxfreq
        except:
            tf = 0
            
        #idf = math.log10((N/Ni)+1)
        try:
            idf = math.log10((self.N/Ni)+1)
        except:
            idf = 0

        return(tf*idf)

    def DictWeight(self):
        self.TDFP = [(word,doc,self.dict[(word,doc)],self.calcWeight(word,doc)) for word,doc in self.dict.keys()]
        self.DTFP = [(doc,word,freq,weight) for word,doc,freq,weight in self.TDFP]
        self.TDFP = sorted(self.TDFP, key=lambda x: x[0])

        
d = Docs(Lancaster=False)

d.DictWeight()

d.TDFP

[('', 0, 1, 0.05376692341046299),
 ('(1)', 0, 1, 0.05376692341046299),
 ('(2)', 0, 1, 0.05376692341046299),
 ('(GNNs).', 0, 1, 0.05376692341046299),
 ('(JNSKR).', 3, 1, 0.05376692341046299),
 ('(KG)', 0, 1, 0.03670163497843557),
 ('(KG)', 3, 1, 0.03670163497843557),
 ('(KGIN).', 0, 1, 0.05376692341046299),
 ('(NS)', 3, 1, 0.05376692341046299),
 ('(TGSRec)', 1, 1, 0.05376692341046299),
 ('(about', 3, 1, 0.05376692341046299),
 ('(e.g.,', 2, 1, 0.03670163497843557),
 ('(e.g.,', 3, 1, 0.03670163497843557),
 ('(i.e.,', 0, 1, 0.05376692341046299),
 ('(including', 3, 1, 0.05376692341046299),
 ('0.70,', 2, 1, 0.05376692341046299),
 ('0.87,', 2, 1, 0.05376692341046299),
 ('0.95', 2, 1, 0.05376692341046299),
 ('20', 3, 1, 0.05376692341046299),
 ('22.1%', 1, 1, 0.05376692341046299),
 ('22.5%', 1, 1, 0.05376692341046299),
 ('A', 0, 1, 0.05376692341046299),
 ('By', 2, 1, 0.05376692341046299),
 ('CKAN', 0, 1, 0.05376692341046299),
 ('Collaborative', 1, 1, 0.05376692341046299),
 ('Empirical', 1, 1, 0

In [118]:
import tkinter as tk
from tkinter import ttk

def search():

    text = query_text.get()
    table.delete(*table.get_children())

    if index_var.get() == 0:
        data = d.TDFP
    else:
        data = d.DTFP

    for terme in data:

        if type(terme[0]) is str:
            if text.lower() in terme[0].lower():
                table.insert("", "end", values=[terme[0],terme[1],terme[2],terme[3]])
        else:
            if text.lower() in terme[1].lower():
                table.insert("", "end", values=[terme[0],terme[1],terme[2],terme[3]])
    # Add your search and processing logic here
    pass

def on_token_change(*args):
    print(processing_var.get())
    if processing_var.get() == 0:
        d.rebuild(Lancaster=False)
    else:
        d.rebuild(Lancaster=True)
    search()

def on_index_change(*args):
    search()

# Create the main application window
root = tk.Tk()
root.title("Text Processing Application")

# Search Query Text Box and Search Button
query_label = tk.Label(root, text="Search Query:")
query_label.grid(row=0, column=0)
query_text = tk.Entry(root, width=40)
query_text.grid(row=0, column=1)
search_button = tk.Button(root, text="Search", command=search)
search_button.grid(row=0, column=2)

# Border around "Processing" section
processing_label_frame = ttk.LabelFrame(root, text="Processing")
processing_label_frame.grid(row=1, column=2, padx=10, pady=10)

# Radio buttons for Processing
processing_var = tk.IntVar()
processing_var.trace("w", on_token_change)

tokenization_radiobutton = tk.Radiobutton(processing_label_frame, text="Tokenization", variable=processing_var, value=0)
porter_stemmer_radiobutton = tk.Radiobutton(processing_label_frame, text="Porter Stemmer", variable=processing_var, value=1)
porter_stemmer_radiobutton.select()
tokenization_radiobutton.grid(row=0, column=0)
porter_stemmer_radiobutton.grid(row=1, column=0)

# Border around "Index" section
index_label_frame = ttk.LabelFrame(root, text="Index")
index_label_frame.grid(row=1, column=0, padx=10, pady=10)

# Radio buttons for Index

index_var = tk.IntVar()
index_var.trace("w", on_index_change)

docs_per_term_radiobutton = tk.Radiobutton(index_label_frame, text="DOCS per TERM", variable=index_var, value=0)
terms_per_doc_radiobutton = tk.Radiobutton(index_label_frame, text="TERMS per DOC", variable=index_var, value=1)
docs_per_term_radiobutton.grid(row=1, column=0)
terms_per_doc_radiobutton.grid(row=2, column=0)

# Table to display data
table_frame = ttk.Frame(root)
table_frame.grid(row=2, column=0, columnspan=3, pady=10)
table = ttk.Treeview(table_frame, columns=("1", "2", "3", "4"))
table.heading("#1", text="terme")
table.heading("#2", text="document")
table.heading("#3", text="freq")
table.heading("#4", text="Score")
table['show'] = 'headings'
# Vertical scrollbar for the table
table_scrollbar = ttk.Scrollbar(table_frame, orient="vertical", command=table.yview)
table.configure(yscrollcommand=table_scrollbar.set)

# Grid layout for the table and scrollbar
table.grid(row=0, column=0, sticky="nsew")
table_scrollbar.grid(row=0, column=1, sticky="ns")


# Make the table resizable
table_frame.grid_rowconfigure(0, weight=1)
table_frame.grid_columnconfigure(0, weight=1)


for dat in d.TDFP:
    table.insert("", "end", values=[dat[0],dat[1],dat[2],dat[3]])


root.mainloop()

1


Exception in Tkinter callback
Traceback (most recent call last):
  File "d:\APPS\python\lib\tkinter\__init__.py", line 1884, in __call__
    return self.func(*args)
  File "C:\Users\Mekki\AppData\Local\Temp\ipykernel_11696\2853912488.py", line 31, in on_token_change
    search()
  File "C:\Users\Mekki\AppData\Local\Temp\ipykernel_11696\2853912488.py", line 7, in search
    table.delete(*table.get_children())
  File "d:\APPS\python\lib\tkinter\ttk.py", line 1225, in get_children
    self.tk.call(self._w, "children", item or '') or ())
_tkinter.TclError: invalid command name ".!frame.!treeview"


0
1
0
1
0
1
0
1
0
1
1
0
1
0
1
0
1
0
1
0
1
0
1
0
1
0
1
0
1
